In [12]:
import gradio as gr
import whisper
from gtts import gTTS
import requests
import tempfile
import base64
import os

In [13]:
#Load environment variables from .env
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

In [14]:
# Load Whisper model
whisper_model = whisper.load_model("base")

In [15]:
def initialize_messages():
    return [{
        "role": "system",
        "content": (
            "You are an AI teacher built for classrooms. Answer with examples, analogies, and help students learn. "
            "Also, suggest relevant images and YouTube video links where appropriate."
            "Respond without using symbols like @, #, %, $, &, *, etc."

        )
    }]

messages_prmt = initialize_messages()

In [16]:
# Convert text to base64 audio
def text_to_base64_audio(text):
    tts = gTTS(text)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        tts.save(temp_audio.name)
        audio_data = open(temp_audio.name, "rb").read()
    os.remove(temp_audio.name)
    return base64.b64encode(audio_data).decode("utf-8")

In [17]:
# Get YouTube video search link
def get_youtube_search_link(query):
    return f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"

In [18]:
# Get Unsplash or DuckDuckGo image search link
def get_image_search_link(query):
    return f"https://duckduckgo.com/?q={query.replace(' ', '+')}&iax=images&ia=images"

In [19]:
# Main function
def voice_teacher(audio_file):
    global messages_prmt

    # Step 1: Transcribe voice
    transcription = whisper_model.transcribe(audio_file)["text"]
    messages_prmt.append({"role": "user", "content": transcription})

    # Step 2: Query OpenRouter
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "deepseek/deepseek-r1-0528:free",
        "messages": messages_prmt
    }
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    result = response.json()

    #Add these two lines immediately after the above
    print("Status code:", response.status_code)
    print("Full response:", result)

    if "choices" not in result:
        return "<p>Sorry, something went wrong with OpenRouter!</p>"

    bot_reply = result["choices"][0]["message"]["content"]
    messages_prmt.append({"role": "assistant", "content": bot_reply})

    # Step 3: Convert to audio
    b64_audio = text_to_base64_audio(bot_reply)

    # Step 4: Extract key topic for image/video
    keyword = transcription.strip().split("?")[0] if "?" in transcription else transcription
    youtube_link = get_youtube_search_link(keyword)
    image_link = get_image_search_link(keyword)

    # Step 5: Return enhanced HTML
    audio_html = f"""
    <p><b>You asked:</b> {transcription}</p>
    <p><b>AI says:</b> {bot_reply}</p>
    <audio autoplay controls style="width: 100%;">
        <source src="data:audio/mp3;base64,{b64_audio}" type="audio/mp3">
        Your browser does not support the audio element.
    </audio>
    <p><b>Related Image Search:</b> <a href="{image_link}" target="_blank">View Images</a></p>
    <p><b>Related Video:</b> <a href="{youtube_link}" target="_blank">Watch on YouTube</a></p>
    """

    return audio_html

In [20]:
# Gradio interface
iface = gr.Interface(
    fn=voice_teacher,
    inputs=gr.Audio(type="filepath", label="Speak your question"),
    outputs=gr.HTML(),
    title="Voice-Enabled AI Teacher",
    description="Speak a question. Get a spoken answer with images and videos!",
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


C:\AI assiss\class_env\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Status code: 200
Full response: {'id': 'gen-1752247705-ZUtCzsEKERAao2KWR6t3', 'provider': 'Chutes', 'model': 'deepseek/deepseek-r1-0528:free', 'object': 'chat.completion', 'created': 1752247705, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'Sure! I\'ll explain a system (like an **Ecosystem**) using simple terms, examples, and analogies—just like I’d teach in a classroom. Let’s break it down step by step:  \n\n---\n\n### 🍃 **What’s a System?**  \nA system is a group of parts that work together as a whole. Imagine a bicycle:  \n- **Parts:** Wheels, pedals, chain, handlebars.  \n- **How they work together:** Pedals turn the chain, which spins the wheels, letting you move.  \nIf one part breaks (e.g., the chain), the whole system fails.  \n\n---\n\n### 🌳 **Example: Forest Ecosystem**  \n**Definition:** A community of plants, animals, and tiny organisms interacting with their environment.  \n\